<a href="https://colab.research.google.com/github/Rajya-3154/tech-a-intern/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:



import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'imdb-movie-ratings-sentiment-analysis:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1875905%2F3063858%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240627%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240627T134631Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D72269028e9e6a2c66c26336978e3dd6e9cd133b5e350e6421e0d4673452020801a2f78e364872c7dfd3fb79f8043cdee24c2d8aab28d5e75d012500abd9c74d134eb0e374f8f470badc49a20b48511ff3b65f0f8ee48279968df328296cab1f457e10bac83e456fdca63ef2e86b8892d963a0a8535c8984005e24421f6e855ef81613aa4af0209753e5647a925338e884f0f44d30b503a1f87e65e587b0e61e4fb817cc36de9986f9125d5680eacc6243ac862e266753c8318878153982e3e58f1f37a9c549521157bf21e5693546ada617393580c49262d6bbf93d471dadad9a5a397d15350c36c59c6f818276ef529154816839d7a6fbc33adbe75ed76f532'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!nvidia-smi

In [ ]:
import pandas as pd
df=pd.read_csv("/kaggle/input/imdb-movie-ratings-sentiment-analysis/movie.csv")
df.head(3)

# label [0 : Negative | 1 : Positive]

In [ ]:
df.info()

In [ ]:
df.label.value_counts()

In [ ]:
display(df.describe())

# Spacy api for Text preprocces

In [ ]:
import spacy
import numpy as np

# load english Large model
nlp = spacy.load("en_core_web_lg")

In [ ]:
# Process the text using spaCy, convert to lowercase, and remove stop words,Punct & conver to Vector
def vector(text):
    doc = nlp(text.lower())
    vectors = [token.vector for token in doc if not token.is_stop and not token.is_punct]
    sentence_vector = np.mean(vectors, axis=0) if vectors else None

    return sentence_vector

In [ ]:
df["vector"]=df['text'].apply(vector)
df.head(3)

In [ ]:
print(df.vector.shape)

In [ ]:
# Split features and Target
feature = np.stack(df.vector.values)
target = df.label.values

In [ ]:
feature.shape,target

In [ ]:
from sklearn.model_selection import train_test_split

# Training and Test data spliting

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.20, random_state=42)
print(X_train.shape,y_train.shape,"|",X_test.shape,y_test.shape)

Model Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

modelLR = LogisticRegression(max_iter=6000,)
modelLR.fit(X_train, y_train)
y_pred = modelLR.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# visualise confusion matrix Logistic Regression

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize = (6,5))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Prediction')
plt.ylabel('Truth')

Trying Deep Learning Models

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Flatten,SimpleRNN,Embedding,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Split features and Target For RNN

feature1 = df.text.values
target1 = df.label.values

In [ ]:
feature1[0]

In [ ]:
# Make New data for taining and testing

X_train_R, X_test_R, y_train_R, y_test_R = train_test_split(feature1, target1, test_size=0.20, random_state=42)
print(X_train_R.shape,y_train_R.shape,"|",X_test_R.shape,y_test_R.shape)

Tokenize Data and Padding for equal input

In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_R)
sequences_train = tokenizer.texts_to_sequences(X_train_R)
sequences_test = tokenizer.texts_to_sequences(X_test_R)
vocab_size = len(tokenizer.word_index) + 1

X_train_R = pad_sequences(sequences_train, padding='post', maxlen=200)
X_test_R = pad_sequences(sequences_test, padding='post', maxlen=200)

In [ ]:
X_train_R[0]

In [ ]:
y_train_R

In [ ]:
X_train_R.shape,y_train_R.shape

# Model building using ANN and Training

In [ ]:
epochs = 20
batch_size=64

modelANN = Sequential()
modelANN.add(Embedding(vocab_size, 300, input_length=200))
modelANN.add(Flatten())
modelANN.add(Dense(130,activation='relu',))
modelANN.add(Dropout(0.5))
modelANN.add(Dense(70,activation='relu',))
modelANN.add(Dropout(0.5))
modelANN.add(Dense(40,activation='relu'))
modelANN.add(Dropout(0.5))
modelANN.add(Dense(1,activation='sigmoid'))
modelANN.summary()
modelANN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history1 = modelANN.fit(X_train_R, y_train_R, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)

In [ ]:
# Visualise Accuracy and Losses

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history1.history['acc'])
plt.plot(history1.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# visualise Classification reports and confusion matrix

y_pred = modelANN.predict(X_test_R)
threshold = 0.5
y_pred_labels = np.where(y_pred >= threshold, 1, 0)

print(classification_report(y_test_R, y_pred_labels))
cm = confusion_matrix(y_test_R, y_pred_labels)
plt.figure(figsize = (3,2))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Prediction')
plt.ylabel('Truth')

# Model building using Simple RNN and Training

In [ ]:
modelRNN = Sequential()
modelRNN.add(Embedding(vocab_size, 100, input_length=200))
modelRNN.add(SimpleRNN(62))
modelRNN.add(Flatten())
modelRNN.add(Dense(22,activation='relu'))
modelRNN.add(Dropout(0.5))
modelRNN.add(Dense(1,activation='sigmoid'))
modelRNN.summary()
modelRNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history2 = modelRNN.fit(X_train_R, y_train_R, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)

In [ ]:
y_pred = modelRNN.predict(X_test_R)
threshold = 0.5
y_pred_labels = np.where(y_pred >= threshold, 1, 0)

print(classification_report(y_test_R, y_pred_labels))
cm = confusion_matrix(y_test_R, y_pred_labels)
plt.figure(figsize = (3,2))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Prediction')
plt.ylabel('Truth')

# Build LSTM Model and Training

In [ ]:
lstm_model = Sequential()
lstm_model.add(Embedding(vocab_size, 300, input_length=200))
lstm_model.add(LSTM(62))
lstm_model.add(Dense(20,activation='relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1,activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history3 = lstm_model.fit(X_train_R, y_train_R, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)

In [ ]:
y_pred = lstm_model.predict(X_test_R)
threshold = 0.5
y_pred_labels = np.where(y_pred >= threshold, 1, 0)

print(classification_report(y_test_R, y_pred_labels))
cm = confusion_matrix(y_test_R, y_pred_labels)
plt.figure(figsize = (3,2))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Prediction')
plt.ylabel('Truth')

In [ ]:
sample_text_pos = ('''Wow! What an incredible movie! The storyline was engaging, the characters were well-
    developed, and the cinematography was stunning. I couldn't take my eyes off the screen
    from start to finish. The performances were outstanding, and the soundtrack added the
    perfect touch. This is a must-watch for any movie enthusiast!''')

sample_text_Neg = ("""I was really disappointed with this movie. The plot was confusing and poorly executed,
and the characters felt one-dimensional. The dialogue was lackluster, and I found it hard
to connect with any of the characters. The pacing was off, and there were too many unnecessary
scenes that didn't contribute to the overall story.I had high expectations, but sadly, this movie
didn't live up to them.""")

tokenizer.fit_on_texts(sample_text_Neg)
sample_text = tokenizer.texts_to_sequences([sample_text_Neg])

sample_text = pad_sequences(sample_text, padding='post', maxlen=100)

predictions = lstm_model.predict(sample_text)

if predictions > 0.5:
    print('The review is positive')
else:
    print('The review is negative')